In [60]:
from urllib.request import urlopen

# открываю текст по url
html = urlopen("http://lib.ru/ZELQZNY/zheliazny_hronomaster.txt")

In [61]:
import re
text = " ".join([re.sub('[^А-Яа-я.!?]', " ", line.decode(html.headers.get_content_charset())) for line in html])

In [62]:
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.tokenize import sent_tokenize, word_tokenize

sentences = [word_tokenize(re.sub('[,:;.!?]', " ", sent.lower())) for sent in sent_tokenize(text)]
n = 2
n_grams, words = padded_everygram_pipeline(n, sentences)

In [65]:
from nltk.lm import MLE

modelMLE = MLE(n)
modelMLE.fit(n_grams, words)
print(f'MLE - {modelMLE.score("замечательно", ["просто"])}')

from nltk.lm import Lidstone

def lidstones(l):
    n_grams_lidstone, words_lidstone = padded_everygram_pipeline(n, sentences)
    model_lidstone = Lidstone(l, n)
    model_lidstone.fit(n_grams_lidstone, words_lidstone)
    print(f'Lidstone лямбда {l} - {model_lidstone.score("замечательно", ["просто"])}')

lidstones(0.1)
lidstones(1)

MLE - 0
Lidstone лямбда 0.1 - 0.0006999236446933062
Lidstone лямбда 1 - 0.0001315529829638887


In [64]:
from nltk.lm import KneserNeyInterpolated
from nltk.lm.preprocessing import pad_both_ends
from nltk.util import ngrams
import math
import random

random.shuffle(sentences)
train_set = sentences[math.floor(len(sentences) * 0.8):]
validation_set = sentences[math.floor(len(sentences) * 0.8):math.floor(len(sentences) * 0.9)]
test_set = sentences[:math.floor(len(sentences) * 0.9)]
best_model = {
    'perplexity': None,
    'n' : None,
    'lambda' : None,
    'name' : None
}

def test_models(model_name: str, l: float, n_size: int, flag_test: bool = False) -> None:
    global perplexity
    n_grams_train, words_train = padded_everygram_pipeline(n_size, train_set)
    current_set = test_set if flag_test else validation_set
    n_grams = [ngrams(pad_both_ends(sent, n_size), n_size) for sent in current_set]
    n_grams_flat = [item for sublist in n_grams for item in sublist]
    print(f'{model_name} лямбда = {l}, n = {n_size}')

    if (model_name == "Lidstone"):
        model_lidstone = Lidstone(l, n_size)
        model_lidstone.fit(n_grams_train, words_train)
        perplexity = model_lidstone.perplexity(n_grams_flat)

    if (model_name == "Kneser-Ney"):
        model_kneserney = KneserNeyInterpolated(n_size, l)
        model_kneserney.fit(n_grams_train, words_train)
        perplexity = model_kneserney.perplexity(n_grams_flat)

    if not(best_model['perplexity']) or perplexity < best_model['perplexity']:
        best_model['perplexity'] = perplexity
        best_model['n'] = n_size
        best_model['lambda'] = l
        best_model['name'] = model_name

    print(perplexity)
    print('--------------------')

n_list = [1, 2, 3, 4, 5]
[test_models('Lidstone', l, n_var) for l in [0.01, 0.05, 0.1, 0.2, 0.5, 1, 2, 5] for n_var in n_list]
[test_models('Kneser-Ney', l, n_var) for l in [0.0, 0.02, 0.05, 0.1, 0.2, 0.5, 0.95, 0.99] for n_var in n_list]

print('Победитель!')
test_models(best_model['name'], best_model['lambda'], best_model['n'], True)

Lidstone лямбда = 0.01, n = 1
5193.825311782082
--------------------
Lidstone лямбда = 0.01, n = 2
1503.5404780230067
--------------------
Lidstone лямбда = 0.01, n = 3
1991.6441580366393
--------------------
Lidstone лямбда = 0.01, n = 4
2159.9373151488
--------------------
Lidstone лямбда = 0.01, n = 5
2218.764583709454
--------------------
Lidstone лямбда = 0.05, n = 1
3557.1005962829813
--------------------
Lidstone лямбда = 0.05, n = 2
1753.8608186768924
--------------------
Lidstone лямбда = 0.05, n = 3
2468.2746450106533
--------------------
Lidstone лямбда = 0.05, n = 4
2637.0226655462334
--------------------
Lidstone лямбда = 0.05, n = 5
2702.346079537439
--------------------
Lidstone лямбда = 0.1, n = 1
3040.534526131875
--------------------
Lidstone лямбда = 0.1, n = 2
1983.9910775091398
--------------------
Lidstone лямбда = 0.1, n = 3
2734.5002797489783
--------------------
Lidstone лямбда = 0.1, n = 4
2896.0618149195825
--------------------
Lidstone лямбда = 0.1, n = 5
29